In [1]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType
import requests
import pandas as pd
from dotenv import load_dotenv
import os

cmf_param = {
    'chain_id': 1,
}

context, _model_loader = create_cmf(cmf_param)

In [2]:
rain_mainnet_factory = Contract('0x389c6f04D39fF038cC99F94BcE338D2f5Eb64a67')
rain_factory_proxy = Contract('0x31EBf70312f488D0bdAc374b340f0D01dBf153B5')

rain_controller = Contract('0xE5D3d7da4b24bc9D2FDA0e206680CD8A00C0FeBD')
usdc_addr = Token("USDC")

load_dotenv()

RAIN_DEVELOPER_API_KEY = os.environ.get('RAIN_DEVELOPER_API_KEY')
RAIN_CARD_SPEND_ENDPOINT = os.environ.get('RAIN_CARD_SPEND_ENDPOINT')


start_date = BlockNumber.from_ymd(2022, 10, 1)
current_date=context.block_number


In [18]:
# rain_mainnet_factory.ledger.events.NewRainCollateral
rain_factory_proxy.ledger.events.NewRainCollateral

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__name', 'evt__user'])

In [3]:
with rain_factory_proxy.ledger.events.NewRainCollateral as q:
    res = q.select(columns=[q.EVT__COLLATERALPROXY],
                    order_by=q.BLOCK_NUMBER.desc(),
                    limit=1000).to_dataframe()
    
print(res)

# iterate through each row in res

# for each row, get the balance of the collateral proxy

# for index, row in res.iterrows():
#     # print(row['evt__collateralProxy'])
#     collateral_proxy = Address(row['evt__collateralProxy'])
#     balance = usdc_addr.functions.balanceOf(collateral_proxy).call()
#     print(balance)
#     # with collateral_proxy.balance_of(usdc_addr) as q:
#     #     res = q.select(columns=[q.EVT__VALUE],
#     #                 order_by=q.BLOCK_NUMBER.desc(),
#     #                 limit=200).to_dataframe()
#     # print(res)
#     # # iterate through each row in res
#     # # for each row, get the balance of the collateral proxy
#     # for index, row in res.iterrows():
#     #     print(row['VALUE'])
#     #     # if the balance is greater than 0, then we need to call the rain card spend endpoint
#     #     if row['VALUE'] > 0:
#     #         print('calling rain card spend endpoint')
#     #         # call the rain card spend endpoint



                          evt__collateralProxy
0   0x961e671268d06b87b4889c0070b5f12321f54221
1   0x06731b3ae3fff7d88956b4e6d0520055ede902c4
2   0x78252d583b636ee4cdfcad2f5ee423adf80bf840
3   0x3d677d45d967b38fcdf9af729b1f5e2df5be676e
4   0xbb3c5287354d26e59503d94259116e395bead52f
5   0x88dbe7cb67dddb7021070d301aba7aa1fd251042
6   0x9c659caf50c3d29ab3b0744a533cffcb5241b2da
7   0xa5b5c417efb8cfb87dd4ab7ffdf5d2d6478e56d0
8   0xe20ddd1e1b0dad8a17d59fd768d726865335d78e
9   0x7d2871b02b87126bbdff8dbf5b6b4cb8226a3036
10  0xc0eafe4ed305f6d7fb1247c5296fab8b450f32d1
11  0xc4ee1ba9a95bb80987ca0a11e9573b4ab5663c54
12  0x196676fb5e8f1b4617443296859623eb4bdb3795
13  0x424866e16698955b202d13795a020ad7ff9ff009
14  0xa7796363b50884fa115ac53809918c011381259a
15  0xbdbc22b35d87bee5d5a584b6ad1b438e14c5f353
16  0xb96ef17babd471c8ba388bd3291c3068f5e5eb47
17  0xe400da55a34f5ea199d1c21dc4224f3f49d71d90
18  0x49dadb85ce88433128243f2bca4f6b6c47955fa4
19  0xd4b6f8a7fe272202d8037cbe2c6ddef491400cea
20  0x2a36ca7

In [75]:
rain_controller.ledger.events.Liquidation

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__assets', 'evt__amounts'])

In [ ]:
with rain_controller.ledger.events.Liquidation as q:
    liquidations = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(liquidations)

In [76]:
rain_controller.ledger.events.Payment

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__assets', 'evt__amounts'])

In [ ]:
with rain_controller.ledger.events.Payment as q:
    payments = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNTS
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(payments)

In [10]:
rain_controller.ledger.events.Withdrawal

LedgerQueryContractEvents(['block_number', 'block_timestamp', 'contract_address', 'log_index', 'signature', 'transaction_hash', 'evt__collateralProxy', 'evt__asset', 'evt__amount'])

In [ ]:
with rain_controller.ledger.events.Withdrawal as q:
    withdrawals = (q.select(
        columns=[
            q.EVT__COLLATERALPROXY,
            q.EVT__AMOUNT
        ],
        where=q.BLOCK_NUMBER.between_(start_date, current_date)
    )).to_dataframe()

display(withdrawals)

In [13]:
def get_card_spend_dataframe():
    # Headers (if required)
    headers = {
        'API-Key': RAIN_DEVELOPER_API_KEY,
        'accept': 'application/json'
    }

    # Make the GET request
    response = requests.get(RAIN_CARD_SPEND_ENDPOINT, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response if successful
        data = response.json()
        print(data)
    else:
        # Handle errors (you can print or log the error response)
        print(f"Error: {response.status_code}")

    # create dataframe
    df = pd.DataFrame([data])

    # add date column
    df.insert(0, 'date', current_date)


    # drop dueBalance
    df = df.drop(columns=['dueBalance'])
    
    # add totalBalance
    df['totalBalance'] = df['pendingBalance'] + df['postedBalance']
    
    return df

In [14]:
latest_card_spend_df = get_card_spend_dataframe()

{'pendingBalance': 3993333, 'postedBalance': 22975207, 'dueBalance': 0}
